In [1]:
from itertools import combinations
from matplotlib import pyplot as plt
from microhapdb import markers

In [2]:
mk = markers.copy()
mk["Locus"] = mk.Name.apply(lambda x: x.split(".")[0])
mk = mk[~mk.Name.str.contains("LV")]
mk = mk.sort_values("Ae", ascending=False)
mk

,Name,NumVars,Extent,Chrom,Start,End,Positions,Positions37,RSIDs,Source,Ae,Locus
237,mh01SCUZJ-0675568,15,248,chr1,246868891,246869138,246868891;246868905;246868927;246868930;246868...,247032193;247032207;247032229;247032232;247032...,NaN,Zhu2023,72.533,mh01SCUZJ-0675568
1056,mh05SCUZJ-0529358,18,273,chr5,180190210,180190482,180190210;180190225;180190275;180190347;180190...,179617210;179617225;179617275;179617347;179617...,NaN,Zhu2023,31.715,mh05SCUZJ-0529358
2444,mh15SCUZJ-0267777,18,242,chr15,100789736,100789977,100789736;100789746;100789749;100789760;100789...,101329941;101329951;101329954;101329965;101329...,NaN,Zhu2023,31.510,mh15SCUZJ-0267777
487,mh03SCUZJ-0000972.v2,16,220,chr3,242309,242528,242309;242313;242318;242322;242325;242341;2423...,283992;283996;284001;284005;284008;284024;2840...,NaN,Zhu2023,28.392,mh03SCUZJ-0000972
828,mh04SCUZJ-0474306,10,329,chr4,128761273,128761601,128761273;128761282;128761319;128761344;128761...,129682428;129682437;129682474;129682499;129682...,NaN,Zhu2023,27.397,mh04SCUZJ-0474306
...,...,...,...,...,...,...,...,...,...,...,...,...
993,mh05SCUZJ-0282942.v2,18,201,chr5,95927626,95927826,95927626;95927652;95927680;95927688;95927693;9...,95263330;95263356;95263384;95263392;95263397;9...,NaN,Zhu2023,1.000,mh05SCUZJ-0282942
992,mh05SCUZJ-0282942.v1,34,350,chr5,95927374,95927723,95927374;95927382;95927394;95927396;95927400;9...,95263078;95263086;95263098;95263100;95263104;9...,NaN,Zhu2023,1.000,mh05SCUZJ-0282942
2669,mh17WL-008.v2,15,285,chr17,80743723,80744007,80743723;80743765;80743766;80743767;80743782;8...,78717523;78717565;78717566;78717567;78717582;7...,rs62067931;rs528918725;rs12941902;rs539712541;...,Yu2022G3,1.000,mh17WL-008
2670,mh17WL-008.v1,11,189,chr17,80743765,80743953,80743765;80743766;80743767;80743782;80743783;8...,78717565;78717566;78717567;78717582;78717583;7...,rs528918725;rs12941902;rs539712541;rs12939914;...,Yu2022G1;Yu2022G2,1.000,mh17WL-008


In [3]:
short = mk[mk.Extent < 100]
medium = mk[(100 <= mk.Extent) & (mk.Extent < 200)]
long = mk[(200 <= mk.Extent) & (mk.Extent < 300)]
print(f"short rows={short.shape[0]} maxAe={short.Ae.iloc[0]}")
print(f"medium rows={medium.shape[0]} maxAe={medium.Ae.iloc[0]}")
print(f"long rows={long.shape[0]} maxAe={long.Ae.iloc[0]}")

short rows=675 maxAe=10.818
medium rows=1139 maxAe=16.593
long rows=768 maxAe=72.533


In [4]:
def best_independent_markers(table, n=40, params=((3, 20e6), (3, 10e6), (2, 20e6), (2, 10e6)), mask=None):
    if mask:
        table = table[~table.Locus.isin(mask)]
    markerids = set()
    for chrom in table.Chrom.unique():
        markers = table[table.Chrom == chrom]
        
        def try_combos(m=3, dist=10e6):
            best_ae, best_markers = None, None
            for ids in combinations(markers.Name, m):
                test_markers = markers[markers.Name.isin(ids)]
                for start1, start2 in combinations(test_markers.Start, 2):
                    if abs(start1 - start2) < dist:
                        break
                else:
                    ae = sum(test_markers.Ae) / len(test_markers.Ae)
                    if best_ae is None or ae > best_ae:
                        best_ae = ae
                        best_markers = ids
            return best_markers
            
        for nn, dist in params:
            ids = try_combos(m=nn, dist=dist)
            if ids is not None:
                markerids.update(ids)
                break
    panel = table[table.Name.isin(markerids)].sort_values("Ae", ascending=False).head(n)
    return set(panel.Name)

In [5]:
panel_short = best_independent_markers(short)
loci_short = set([m.split(".")[0] for m in panel_short])

In [6]:
panel_medium = best_independent_markers(medium, mask=loci_short)
loci_medium = set([m.split(".")[0] for m in panel_medium])
assert loci_short & loci_medium == set()

In [7]:
panel_long = best_independent_markers(long, mask=loci_short | loci_medium)
loci_long = set([m.split(".")[0] for m in panel_long])
assert loci_short & loci_long == set()
assert loci_medium & loci_long == set()

In [8]:
panel = panel_short | panel_medium | panel_long

In [9]:
d = mk[mk.Name.isin(panel)].sort_values(["Chrom", "Start"])
d

,Name,NumVars,Extent,Chrom,Start,End,Positions,Positions37,RSIDs,Source,Ae,Locus
25,mh01SCUZJ-0041531,11,270,chr1,10175409,10175678,10175409;10175422;10175434;10175442;10175454;1...,10235467;10235480;10235492;10235500;10235512;1...,NaN,Zhu2023,13.895,mh01SCUZJ-0041531
105,mh01WL-033.v1,6,89,chr1,101574890,101574978,101574890;101574894;101574897;101574911;101574...,102040446;102040450;102040453;102040467;102040...,rs12562987;rs7544172;rs79314878;rs78437080;rs7...,Yu2022G1;Yu2022G2;Yu2022G4,5.361,mh01WL-033
137,mh01WL-010.v1,7,187,chr1,168484004,168484190,168484004;168484059;168484091;168484099;168484...,168453242;168453297;168453329;168453337;168453...,rs10918905;rs74120927;rs72703787;rs11576829;rs...,Yu2022G1;Yu2022G2,10.860,mh01WL-010
163,mh01KK-212.v5,6,99,chr1,202647461,202647559,202647461;202647462;202647488;202647509;202647...,202616589;202616590;202616616;202616637;202616...,rs57658975;rs72750624;rs11589785;rs58023111;rs...,Yu2022G4,8.672,mh01KK-212
177,mh01WL-005.v1,5,169,chr1,212172159,212172327,212172159;212172178;212172241;212172286;212172327,212345501;212345520;212345583;212345628;212345669,rs11119867;rs11119868;rs4951574;rs11583051;rs7...,Yu2022G1;Yu2022G2,16.593,mh01WL-005
...,...,...,...,...,...,...,...,...,...,...,...,...
1564,mh09USC-9pA.v1,5,72,chr9,2288647,2288718,2288647;2288677;2288700;2288710;2288718,2288647;2288677;2288700;2288710;2288718,rs1408329;rs11789647;rs12555748;rs1535838;rs14...,delaPuente2020,5.388,mh09USC-9pA
1581,mh09SCUZJ-0076471,10,199,chr9,12356407,12356605,12356407;12356432;12356442;12356447;12356455;1...,12356407;12356432;12356442;12356447;12356455;1...,NaN,Zhu2023,10.883,mh09SCUZJ-0076471
1596,mh09SHY-001.v3,6,290,chr9,28516884,28517173,28516884;28517003;28517012;28517139;28517172;2...,28516882;28517001;28517010;28517137;28517170;2...,rs10968631;rs10491886;rs28472295;rs10491885;rs...,Yu2022G3,11.058,mh09SHY-001
1628,mh09KK-153.v4,4,149,chr9,101207458,101207606,101207458;101207493;101207570;101207606,103969740;103969775;103969852;103969888,rs10125791;rs2987741;rs7047561;rs1156731,Yu2022G1;Yu2022G2;Yu2022G3,7.334,mh09KK-153


In [17]:
for chrom in sorted(d.Chrom.unique()):
    cmarkers = d[d.Chrom == chrom]
    least_dist = float("Inf")
    least_dist_markers = None
    for (n, marker1), (m, marker2) in combinations(cmarkers.iterrows(), 2):
        dist = abs(marker1.Start - marker2.Start)
        if dist < 10e6:
            print(f"{chrom:8s}: {dist:8d} {marker1.Name}, {marker2.Name}")
        #if dist < least_dist:       
        #    least_dist = dist
        #    least_dist_markers = (marker1.Name, marker2.Name)
    #print(f"{chrom}: {least_dist} bp {least_dist_markers}")

chr1    :  9524698 mh01KK-212.v5, mh01WL-005.v1
chr1    :  5167456 mh01WL-006.v1, mh01WL-008.v3
chr1    :  5182621 mh01WL-008.v3, mh01SCUZJ-0675568
chr10   :   641339 mh10SCUZJ-0020886, mh10WL-031
chr10   :  1075056 mh10SCUZJ-0020886, mh10WL-012.v2
chr10   :   433717 mh10WL-031, mh10WL-012.v2
chr10   :  5212231 mh10SHY-002.v2, mh10SCUZJ-0380360
chr10   :  2372096 mh10WL-001.v1, mh10WL-042
chr11   :  8657032 mh11WL-033.v1, mh11KK-183.v3
chr13   :  6494641 mh13SCUZJ-0045733, mh13USC-13qA.v4
chr13   :  6854185 mh13WL-001.v3, mh13KK-221.v1
chr14   :  5464965 mh14WL-036.v3, mh14SCUZJ-0040957.v1
chr14   :  6069564 mh14SHY-002.v2, mh14SHY-003.v2
chr15   :  1483740 mh15SHY-004.v2, mh15SCUZJ-0251770.v2
chr15   :  3945294 mh15SHY-004.v2, mh15SCUZJ-0267777
chr15   :  2461554 mh15SCUZJ-0251770.v2, mh15SCUZJ-0267777
chr16   :  1006316 mh16KK-259.v6, mh16SCUZJ-0296189
chr16   :  4787085 mh16KK-259.v6, mh16WL-038
chr16   :  3780769 mh16SCUZJ-0296189, mh16WL-038
chr17   :  3140378 mh17WL-022.v1, mh17W

In [10]:
d.Ae.quantile([0, 0.25, 0.5, 0.75, 1])

0.00     5.09000
0.25     6.51925
0.50     9.26400
0.75    13.02850
1.00    72.53300
Name: Ae, dtype: float64

In [11]:
d.Source.apply(lambda x: x.split(";")).explode().value_counts()

Source
Zhu2023           52
Yu2022G1          27
Yu2022G2          20
Yu2022G4          19
Yu2022G3          13
Zhang2023          8
NimaGen2023        5
Pakstis2021        4
Gandotra2020       3
Wu2021             3
delaPuente2020     3
Staadig2021        2
Name: count, dtype: int64